## Dense IR
In this notebook, we show how to train a model, index data, and run search using Neural IR  : in this case ColBERT v2.0 (Santhanam et. al, 2022). TODO: add dependencies

In orded to run (almost) instantaneously, we use trivial data sizes of training data and collection to search.

## Dependencies
If not already done, make sure to install PrimeQA with notebooks extras before getting started.

In [1]:
# If you want CUDA 11 uncomment and run this (for CUDA 10 or CPU you can ignore this line).
#! pip install 'torch~=1.11.0' --extra-index-url https://download.pytorch.org/whl/cu113

# Uncomment to install PrimeQA from source (pypi package pending).
# The path should be the project root (e.g. '.' below).
#! pip install .[notebooks]

## Configuration
First, we need to include the required modules.

In [1]:
import os
import tempfile
 
from primeqa.ir.dense.colbert_top.colbert.utils.utils import create_directory, print_message
from primeqa.ir.dense.colbert_top.colbert.infra import Run, RunConfig
from primeqa.ir.dense.colbert_top.colbert.infra.config import ColBERTConfig
from primeqa.ir.dense.colbert_top.colbert.training.training import train
from primeqa.ir.dense.colbert_top.colbert.indexing.collection_indexer import encode
from primeqa.ir.dense.colbert_top.colbert.searcher import Searcher

No CUDA runtime is found, using CUDA_HOME='/opt/share/cuda-11.8'
{"time":"2023-07-05 09:28:02,108", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss."}
{"time":"2023-07-05 09:28:02,124", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss."}


## Training
We will train a ColBERT model using a TSV file containing [query, positive document, negative document] triples. We use the XOR-TyDi dataset, as described here: https://nlp.cs.washington.edu/xorqa/

The path in `test_files_location` below points to the location of files used by the notebook, by default it points to the files used by CI testing.

In [2]:
test_files_location = '../../../tests/resources/ir_dense'
model_type = 'xlm-roberta'
model_name_or_path = 'xlm-roberta-base'
with tempfile.TemporaryDirectory() as working_dir:
    output_dir=os.path.join(working_dir, 'output_dir')
text_triples_fn = os.path.join(test_files_location, "xorqa.train_ir_negs_5_poss_1_001pct_at_0pct.tsv")

Here is an example of a training file record:

In [3]:
import pandas as pd
from IPython.display import display, HTML
data = pd.read_csv(text_triples_fn, sep='\t', nrows=1, header=None)
display(HTML(data.to_html()))

,0,1,2
0,중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?,"Kangxi Emperor The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang.","Chiddy Bang new songs from the duo and in November 2009 debuted the group's first free mixtape entitled ""The Swelly Express"". On 28 April 2011 during the first-ever MTV O Music Awards, Anamege broke the Guinness World Record for Longest Freestyle Rap and Longest Marathon Rapping Record by freestyling for 9 hours, 18 minutes, and 22 seconds, stealing the throne from rapper M-Eighty, who originally broke the record in 2009 by rapping for 9 hours, 15 minutes and 15 seconds. Anamege had also beat Canadian rapper D.O. for Longest Marathon Rapping session, the previous record being for 8 hours and 45 minutes."


In [4]:
args_dict = {
                'root': output_dir,
                'experiment': 'test_training',
                'triples': text_triples_fn,
                'model_type': model_type,
                'checkpoint': model_name_or_path,
                'maxsteps': 3,
                'bsize': 1,
                'accumsteps': 1,
                'epochs': 1,
                'rank': 0,
                'nranks': 1,
                'amp' : True
            }

Next we train the model, and save it's location in `the latest_model_fn`variable

In [5]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    latest_model_fn = train(colBERTConfig, text_triples_fn, None, None)

{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "index_location": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "cosine",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta",
    "local_models_repository": null,
    "ranks_fn": null,
    "output_dir": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta",
    "teacher_doc_maxlen": 180,
    "disti

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Jul 05, 09:28:53] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jul 05, 09:28:54] 1 epochs of 5 examples
[Jul 05, 09:28:54] batch size: 1
[Jul 05, 09:28:54] maxsteps set to 3
[Jul 05, 09:28:54] start batch idx: 0
[Jul 05, 09:28:54] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[Jul 05, 09:28:54] #> Input: $ 중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?, 		 True, 		 None
[Jul 05, 09:28:54] #> Output IDs: torch.Size([32]), tensor([     0,   9748,  24120,   1180,  13968, 211059,  83639,  76826,  78363,
         57104,    993, 161732,    697, 116932, 114150,     32,      2,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[Jul 05, 09:28:54] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[Jul 05, 09:28:54] #> XLMR DocTokenizer.te

/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Jul 05, 09:28:54] #>>>> before linear query ==
[Jul 05, 09:28:54] #>>>>> Q: torch.Size([32, 768]), tensor([[ 0.1269,  0.0868,  0.0588,  ..., -0.0978,  0.0423,  0.0024],
        [ 0.0237,  0.0695,  0.0303,  ..., -0.0671, -0.0818, -0.0653],
        [ 0.0626,  0.0780,  0.0324,  ...,  0.0213, -0.0027,  0.1598],
        ...,
        [ 0.1314,  0.0969,  0.0671,  ..., -0.1059,  0.0561, -0.0338],
        [ 0.0398,  0.0168,  0.0476,  ..., -0.0210, -0.0167, -0.0480],
        [ 0.0871,  0.0832,  0.0208,  ..., -0.0546,  0.0003,  0.0225]],
       grad_fn=<SelectBackward0>)
[Jul 05, 09:28:54] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2613e-02, -3.6824e-03, -1.0773e-02,  ..., -1.3896e-02,
          1.2244e-02, -8.2326e-03],
        [-3.4988e-03,  3.9623e-03, -2.4905e-02,  ..., -2.6214e-02,
          4.0837e-03, -8.0812e-03],
        [-1.1158e-02, -2.1660e-05,  1.5019e-02,  ..., -1.1310e-02,
         -9.3691e-03, -2.2202e-02],
        ...,
        [-1.0295e-02,  3.2577e-02,  3.3341

## Indexing
Next, we will index a collection of documents, using model representaion from the previous step. 
The collection is a TSV file, containing each document's ID, title, and text.

In [6]:
collection_fn = os.path.join(test_files_location, "xorqa.train_ir_001pct_at_0_pct_collection_fornum.tsv")

Here is an example document:

In [7]:
data = pd.read_csv(collection_fn, sep='\t', header=0, nrows=1)
display(HTML(data.to_html()))

,id,text,title
0,1,"The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang.",Kangxi Emperor


Here are the indexer arguments:

In [9]:
args_dict = {
                'root': os.path.join(output_dir,'test_indexing'),
                'experiment': 'test_indexing',
                'model_type': model_type,
                'checkpoint': latest_model_fn,
                'collection': collection_fn,
                'index_root': os.path.join(output_dir, 'test_indexing', 'indexes'),
                'index_name': 'index_name',
                'doc_maxlen': 180,
                'num_partitions_max': 2,
                'kmeans_niters': 1,
                'nway': 1,
                'rank': 0,
                'nranks': 1,
                'amp' : True
            }

Here we run the indexer:

In [10]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    create_directory(colBERTConfig.index_path_)
    encode(colBERTConfig, collection_fn, None, None)




[Jul 05, 00:49:50] #> Creating directory /tmp/tmp15kb94n1/output_dir/test_indexing/indexes/index_name 


{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "index_location": null,
    "nbits": 1,
    "kmeans_niters": 1,
    "num_partitions_max": 2,
    "similarity": "cosine",
    "bsize": 32,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 500000,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 1,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 10,
    "input_arguments": {},
    "model_type": "xlm-roberta",
    "local_models_repository": null,
    "ranks_fn": null,
    "output_dir": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher

/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Jul 05, 00:50:01] #>>>> before linear doc ==
[Jul 05, 00:50:01] #>>>>> D: torch.Size([180, 768]), tensor([[ 7.2345e-02,  9.6544e-02,  5.4753e-02,  ..., -1.4623e-01,
          5.4647e-02, -7.2017e-05],
        [ 6.8214e-03,  2.6334e-02,  3.3698e-02,  ..., -1.3284e-01,
          1.2795e-02,  1.1620e-01],
        [-4.2909e-02,  9.3319e-02, -3.2824e-02,  ..., -2.9806e-01,
         -2.8376e-02,  9.0276e-02],
        ...,
        [-5.2570e-02,  5.3625e-02,  4.5890e-03,  ..., -1.7077e-01,
         -4.3442e-02,  4.8794e-03],
        [-5.2570e-02,  5.3625e-02,  4.5890e-03,  ..., -1.7077e-01,
         -4.3442e-02,  4.8794e-03],
        [-5.2570e-02,  5.3625e-02,  4.5890e-03,  ..., -1.7077e-01,
         -4.3442e-02,  4.8794e-03]])
[Jul 05, 00:50:01] #>>>>> self.linear doc : Parameter containing:
tensor([[-1.2618e-02, -3.6898e-03, -1.0781e-02,  ..., -1.3904e-02,
          1.2236e-02, -8.2367e-03],
        [-3.4999e-03,  3.9670e-03, -2.4897e-02,  ..., -2.6206e-02,
          4.0920e-03, -8.0773e-03

0it [00:00, ?it/s]

[Jul 05, 00:50:01] [0] 		 #> Encoding 7 passages..
[Jul 05, 00:50:02] [0] 		 #> Saving chunk 0: 	 7 passages and 1,222 embeddings. From #0 onward.


1it [00:00,  1.19it/s]

[Jul 05, 00:50:02] offset: 0
[Jul 05, 00:50:02] chunk codes size(0): 1222
[Jul 05, 00:50:02] codes size(0): 1222
[Jul 05, 00:50:02] codes size(): torch.Size([1222])
[Jul 05, 00:50:02] >>>>partition.size(0): 2
[Jul 05, 00:50:02] >>>>num_partition: 2
[Jul 05, 00:50:02] #> Optimizing IVF to store map from centroids to list of pids..
[Jul 05, 00:50:02] #> Building the emb2pid mapping..
[Jul 05, 00:50:02] len(emb2pid) = 1222



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 5207.08it/s]

[Jul 05, 00:50:02] #> Saved optimized IVF to /tmp/tmp15kb94n1/output_dir/test_indexing/indexes/index_name/ivf.pid.pt
[Jul 05, 00:50:02] [0] 		 #> Saving the indexing metadata to /tmp/tmp15kb94n1/output_dir/test_indexing/indexes/index_name/metadata.json ..


The resulting index files are in `output_dir/test_indexing/indexes/index_name`

## Search
Next, we use the trained model and the index to search the collection, using queries from a TSV query file.

In [11]:
queries_fn = os.path.join(test_files_location, "xorqa.train_ir_001pct_at_0_pct_queries_fornum.tsv")

Here are the search arguments:

In [12]:
args_dict = {
                'root': output_dir,
                'experiment': 'test_indexing' ,
                'checkpoint': latest_model_fn,
                'model_type': model_type,
                'index_location': os.path.join(output_dir, 'test_indexing', 'indexes', 'index_name'),
                'queries': queries_fn,
                'bsize': 1,
                'topK': 1,
                'nway': 1,
                'rank': 0,
                'nranks': 1,
                'amp': True,
            }

Here we initalize and run the searcher:

In [13]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    searcher = Searcher(args_dict['index_location'], checkpoint=args_dict['checkpoint'], config=colBERTConfig)
    rankings = searcher.search_all(args_dict['queries'], args_dict['topK'])

[Jul 05, 00:50:18] #> base_config.py from_path /tmp/tmp15kb94n1/output_dir/test_indexing/indexes/index_name/metadata.json
[Jul 05, 00:50:18] #> base_config.py from_path args loaded! 
[Jul 05, 00:50:18] #> base_config.py from_path args replaced ! 
[Jul 05, 00:50:18] #> base_config.py load_from_checkpoint /tmp/tmp15kb94n1/output_dir/test_training/2023-07/05/00.47.56/checkpoints/colbert
[Jul 05, 00:50:18] #> base_config.py load_from_checkpoint /tmp/tmp15kb94n1/output_dir/test_training/2023-07/05/00.47.56/checkpoints/colbert/artifact.metadata
[Jul 05, 00:50:18] #> base_config.py from_path /tmp/tmp15kb94n1/output_dir/test_training/2023-07/05/00.47.56/checkpoints/colbert/artifact.metadata
[Jul 05, 00:50:18] #> base_config.py from_path args loaded! 
[Jul 05, 00:50:18] #>>>>> at ColBERT name (model name) : /tmp/tmp15kb94n1/output_dir/test_training/2023-07/05/00.47.56/checkpoints/colbert
[Jul 05, 00:50:18] #>>>>> at BaseColBERT name (model name) : /tmp/tmp15kb94n1/output_dir/test_training/2023-

/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Jul 05, 00:50:30] #>>>> before linear query ==
[Jul 05, 00:50:30] #>>>>> Q: torch.Size([32, 768]), tensor([[ 0.1819,  0.1031,  0.0650,  ..., -0.1309,  0.0453, -0.0159],
        [ 0.0719,  0.0487,  0.0614,  ..., -0.0722, -0.1227, -0.0629],
        [ 0.1158,  0.0483,  0.0138,  ...,  0.0008, -0.0050,  0.2066],
        ...,
        [ 0.1729,  0.0918, -0.0352,  ..., -0.2918, -0.0818,  0.0493],
        [ 0.1729,  0.0918, -0.0352,  ..., -0.2918, -0.0818,  0.0493],
        [ 0.1729,  0.0918, -0.0352,  ..., -0.2918, -0.0818,  0.0493]])
[Jul 05, 00:50:30] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2618e-02, -3.6898e-03, -1.0781e-02,  ..., -1.3904e-02,
          1.2236e-02, -8.2367e-03],
        [-3.4999e-03,  3.9670e-03, -2.4897e-02,  ..., -2.6206e-02,
          4.0920e-03, -8.0773e-03],
        [-1.1149e-02, -2.6172e-05,  1.5013e-02,  ..., -1.1318e-02,
         -9.3766e-03, -2.2203e-02],
        ...,
        [-1.0288e-02,  3.2584e-02,  3.3274e-03,  ...,  3.1873e-02,
         

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 41.58it/s]


Here is the search result for our query, containing [query_id, document_id, rank, score]:

In [14]:
rankings.flat_ranking[0]

('-7239279093922981232', 2, 1, 30.169570922851562)

Here is the text of the query record, contataing the ID and text of the query:

In [15]:
with open(queries_fn, 'r') as f:
    for line in f.readlines():
        if str(rankings.flat_ranking[0][0]) == line.split()[0]:
            print(line)

-7239279093922981232	중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?



English translation: `Who maintained the throne for the longest time in China?`

Here is the top retrieved document:

In [16]:
with open(collection_fn, 'r') as f:
    for line in f.readlines():
        if str(rankings.flat_ranking[0][1]) == line.split()[0]:
            print(line)

2	Yao. The Bamboo Annals says that when Emperor Zhuanxu died, a descendent of Shennong named ShuQe raised a disturbance, but was destroyed by the prince of Sin, who was Ku (GaoXin), a descendant of HuangDi, who then ascended to the throne. In the 45th year, Ku designated the prince of Tang (唐) (his son Yao) as his successor, however upon his death in the 63rd year, his elder son Zhi then took the throne instead, ruling nine years before being deposed and replaced by Yao. Emperor Zhi Di Zhì () or simply Zhì, was a mythological emperor of ancient China.	Emperor Zhi

